# Create Zip Code Level Summary Dataset

In [1]:
import os 
from datetime import datetime
import pandas as pd
import gmaps
import apikeys
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('bmh')

# Configure gmaps
gmaps.configure(api_key=apikeys.gmap_key)
%matplotlib inline  

In [2]:
#Read in Data
path = os.path.join('..', 'Resources', 'project_data.csv')
solar = pd.read_csv(path)

C:\Users\bodej\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Parse lat/long from Location 1 variable
latlong = pd.DataFrame(solar["Location 1"].str.split('(',1).tolist(),
                                   columns = ['junk','realstuff'])
latlong['realstuff'] = latlong.realstuff.str.replace(  ')', '')
latlong = latlong.realstuff.str.split(", ", expand = True )
solar['lat'] = latlong.iloc[:, 0]
solar['long'] = latlong.iloc[:, 1]
solar['lat'] = pd.to_numeric(solar['lat'])
solar['long'] = pd.to_numeric(solar['long'])

del latlong


In [4]:
#04 Produce a dataset summarizing key variables by zip code
solar_byzip_group = solar.groupby(by = ["Zip Code"])
solar_byzip = pd.DataFrame(solar_byzip_group.mean())
print(solar_byzip.head())

            Unnamed: 0   Project Cost    $Incentive  Total Nameplate kW DC  \
Zip Code                                                                     
501       52608.416667   99545.309167   9133.083333                31.5575   
10001     23612.400000   75003.056000  17103.000000                 9.4840   
10002     39548.000000   47698.800000  15107.200000                 6.5580   
10009     32691.600000   53099.400000  10138.000000                10.9400   
10011     41404.000000  320921.577500  48147.125000                43.1200   

          Expected KWh Annual Production  $_per_watt  $_per_watt_wincentives  \
Zip Code                                                                       
501                         37040.916667    4.523585                4.311025   
10001                       11131.400000    8.149811                6.467862   
10002                        7696.800000    7.381365                5.037903   
10009                       12841.800000    8.394005 

In [5]:
#Rename variables
solar_byzip.rename(inplace = True, 
             columns = { 
                 "Project_cost": "Avg_cost", 
                 "$Incentive":"Avg_incentive",
                 "Expected KWh Annual Production": "avg_annualkwh", 
                 "$_per_watt": "Avg_$perwatt",
                 "$_per_watt_wincentives": "Avg_$perwatt_wincentive",
                 "Total Nameplate kW DC": "Avg_installedkW"
             })
                   
#Add in calculated variables
solar_byzip['Total_installedkW'] = solar_byzip_group['Total Nameplate kW DC'].sum()
solar_byzip['Total_AnnualMWh'] = solar_byzip_group['Expected KWh Annual Production'].sum()/1000
solar_byzip['Projects'] = solar_byzip_group['Project Number'].count()
solar_byzip['solar_per_000'] = solar_byzip['Total_installedkW']/(solar_byzip['population']/1000)

In [6]:
solar_byzip['solar_per_000'].describe()
solar_byzip.to_pickle("../Resources/Solar data summary by zip")

In [7]:
check = solar_byzip[[ 'solar_per_000', 'population', 'Total_installedkW']]
check.head()

,solar_per_000,population,Total_installedkW
Zip Code,,,
501,NaN,NaN,378.69
10001,4.317582,21966.0,94.84
10002,0.398949,82191.0,32.79
10009,0.870880,62810.0,54.70
10011,3.306305,52167.0,172.48


In [8]:
#Create location lat/long combos
import numpy as np
solar_byzip.replace([np.inf, -np.inf], np.nan, inplace=True)
solar_byzip.dropna(how = 'any', inplace = True)
locations = solar_byzip[["lat", "long"]]

#Options
solar_byzip['solar_weight'] = solar_byzip.solar_per_000.rank(pct = True)*8.9 + 1

# Plot Base map              
fig = gmaps.figure(map_type="TERRAIN", zoom_level=7, center=(42.9, -75))


# Create heat layer
heat_layer = gmaps.heatmap_layer(
    solar_byzip[["lat", "long"]], dissipating=False, weights= solar_byzip['solar_weight'], 
    max_intensity=50, point_radius=0.10, opacity = 0.4)

fig.add_layer(heat_layer)
                                                        
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [22]:
heat_layer.max_intensity =50
heat_layer.point_radius= 0.10

In [23]:
# Plot Base map     
from ipywidgets.embed import embed_minimal_html
fig = gmaps.figure(map_type="TERRAIN", zoom_level=7, center=(42.9, -75))

#Info Box content - Iterate through row, cresolar_byzip['solar_per_000']te string and append to pop up list
popup_list = []

for index, row in solar_byzip.iterrows():
    popup = f"Zip Code: {index}<br>\n" \
            f"Solar per 1,000 (kW): {row['solar_per_000']:.2f}<br>\n" \
            f"Population: {row['population']:,.0f}<br>\n" \
            f"Total solar (kW): {row['Total_installedkW']:,.0f}"
            
    popup_list.append(popup) 



#Create symbol layer (with hover and popups)
solar_byzip['solar_weight'] = solar_byzip.solar_per_000.rank(pct = True)*8.9 + 1
symbol_layer = gmaps.symbol_layer(
    locations, fill_color = 'rgba(0, 152, 215, 0.4)',
    stroke_color = 'rgba(0, 152, 215, 0.2)', 
    scale = [int(x) for x in solar_byzip.solar_weight.values],
    info_box_content = popup_list
)

# #Add layers
# fig.add_layer(heat_layer)
fig.add_layer(symbol_layer)

embed_minimal_html('Solar Concentration.html', views=[fig])

fig

Figure(layout=FigureLayout(height='420px'))